In [2]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

from mordred import Calculator, descriptors

# Thông tin ban đầu về bộ dữ liệu

## 1. Đọc dữ liệu

In [4]:
all_data_path = "../../data_for_modeling/raw_data/all_data/HDAC2_ALL_DATA.xlsx"
dataset = pd.read_excel(all_data_path, sheet_name='original_data')

In [6]:
print(len(dataset))
dataset.head()

4086


,CID,SMILES,IC50(uM),Activity
0,264,CCCC(=O)O,12,Unspecified
1,2662,CCCC(=O)O,NaN,Inconclusive
2,2746,CC(=O)NC1=CC=C(C=C1)C(=O)NC2=CC=CC=C2N,3.81,Active
3,2746,CC(=O)NC1=CC=C(C=C1)C(=O)NC2=CC=CC=C2N,1.2,Active
4,2746,CC(=O)NC1=CC=C(C=C1)C(=O)NC2=CC=CC=C2N,6,Active


In [7]:
def show_activity_distribution(label, dataset):
    #Rows for specific labels
    active_rows = dataset.loc[dataset[label] == "Active"]
    inactive_rows = dataset.loc[dataset[label] == "Inactive"]
    inconclusive_rows = dataset.loc[dataset[label] == "Inconclusive"]
    unspecified_rows = dataset.loc[dataset[label] == "Unspecified"]
    dataset_length = len(dataset)
    print("Total dataset")
    table = [['', 'Active', 'Inactive', 'Inconclusive', 'Unspecified'], 
            ['Number', len(active_rows), len(inactive_rows), len(inconclusive_rows), len(unspecified_rows)],
            ['Percentage (%)', len(active_rows)/dataset_length*100, len(inactive_rows)/dataset_length*100,
            len(inconclusive_rows)/dataset_length*100, len(unspecified_rows)/dataset_length*100]]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

## 2. Phân bố Activity

In [8]:
show_activity_distribution(dataset=dataset, label='Activity')

Total dataset
╒════════════════╤═══════════╤════════════╤════════════════╤═══════════════╕
│                │    Active │   Inactive │   Inconclusive │   Unspecified │
╞════════════════╪═══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 2873      │  25        │      117       │     1071      │
├────────────────┼───────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │   70.3133 │   0.611845 │        2.86344 │       26.2115 │
╘════════════════╧═══════════╧════════════╧════════════════╧═══════════════╛


## 3. IC50 lỗi

In [9]:
def find_non_float_ic50(dataset, ic50_col_name):
    # Use to_numeric to identify non-float values
    is_float = pd.to_numeric(dataset[ic50_col_name], errors='coerce').notna()
    # Find the rows where 'IC50 (uM)' is not a float
    non_float_rows = dataset[~is_float]
    return non_float_rows

In [10]:
non_float_rows = find_non_float_ic50(dataset, 'IC50(uM)')
print(len(non_float_rows))
non_float_rows.head()

853


,CID,SMILES,IC50(uM),Activity
1,2662,CCCC(=O)O,NaN,Inconclusive
16,3810,CN(C)C1=CC=C(C=C1)C(=O)NCCCCC(=O)NO,>10,Unspecified
17,3810,CN(C)C1=CC=C(C=C1)C(=O)NCCCCC(=O)NO,>10,Unspecified
18,3811,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCC(=O)NO,>10,Unspecified
26,4261,C1=CC=C(C(=C1)N)NC(=O)C2=CC=C(C=C2)CNC(=O)OCC3...,NaN,Inconclusive


In [11]:
show_activity_distribution(dataset=non_float_rows, label='Activity')

Total dataset
╒════════════════╤══════════╤════════════╤════════════════╤═══════════════╕
│                │   Active │   Inactive │   Inconclusive │   Unspecified │
╞════════════════╪══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │        0 │   8        │        98      │      747      │
├────────────────┼──────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │        0 │   0.937866 │        11.4889 │       87.5733 │
╘════════════════╧══════════╧════════════╧════════════════╧═══════════════╛


Các xử lý IC50 với từng loại nhãn
- Active: nếu chứa (<, >), ta sẽ loại bỏ những dấu này đi, nếu chất đó là ô trống, hoặc bằng 0 thì ta loại bỏ.
- Inactive: nếu chứa (<, >), ta sẽ loại bỏ những dấu này đi, nếu chất đó là ô trống, hoặc bằng 0 thì ta loại bỏ.
- Inconclusive: những chất này thường có IC50 bị lỗi, ta loại thẳng những chất này đi.
- Unspecified: Những chất này sẽ được chuyển sang nhãn mới là Inactive, tuy nhiên, những chất không có IC50 vẫn bị loại bỏ.

## 4. Nhãn bị mâu thuẫn

In [12]:
def check_label_intersection(dataset, col_name):
    active_rows = dataset.loc[dataset[col_name] == "Active"]
    inactive_rows = dataset.loc[dataset[col_name] == "Inactive"]
    inconclusive_rows = dataset.loc[dataset[col_name] == "Inconclusive"]
    unspecified_rows = dataset.loc[dataset[col_name] == "Unspecified"]
    
    cid_active = active_rows.loc[:, 'CID']
    cid_inactive = inactive_rows.loc[:, 'CID']
    cid_incon = inconclusive_rows.loc[:, 'CID']
    cid_unspec = unspecified_rows.loc[:, 'CID']

    ac_inac_cid = np.intersect1d(cid_active, cid_inactive)
    ac_incon_cid = np.intersect1d(cid_active, cid_incon)
    ac_unspec_cid = np.intersect1d(cid_active, cid_unspec)

    inac_incon_cid = np.intersect1d(cid_inactive, cid_incon)
    incon_unspec_cid = np.intersect1d(cid_incon, cid_unspec)
    inac_unspec_cid = np.intersect1d(cid_inactive, cid_unspec)
    
    print("Activity intersection:")
    table = [['Active-Inactive', 'Active-Inconclusive', 'Active-Unspecified', 'Inactive-Inconclusive', 'Inactive-Unspecified', 'Inconclusive-Unspecifid'], 
             [len(ac_inac_cid), len(ac_incon_cid), len(ac_unspec_cid), len(inac_incon_cid), len(inac_unspec_cid), len(incon_unspec_cid)]]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [13]:
check_label_intersection(dataset=dataset, col_name='Activity')

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 0 │                    13 │                   74 │                       2 │                      2 │                         5 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


# Loại bỏ, ghép chất và tính trung bình từng chất

## Loại bỏ các chất Inconclusive

In [14]:
dataset_c = dataset.copy()
dataset_c.head()

,CID,SMILES,IC50(uM),Activity
0,264,CCCC(=O)O,12,Unspecified
1,2662,CCCC(=O)O,NaN,Inconclusive
2,2746,CC(=O)NC1=CC=C(C=C1)C(=O)NC2=CC=CC=C2N,3.81,Active
3,2746,CC(=O)NC1=CC=C(C=C1)C(=O)NC2=CC=CC=C2N,1.2,Active
4,2746,CC(=O)NC1=CC=C(C=C1)C(=O)NC2=CC=CC=C2N,6,Active


In [15]:
def drop_by_activity(dataset, activity_type, col_name):
    drop_rows = dataset.loc[dataset[col_name] == activity_type]
    dataset.drop(drop_rows.index, inplace=True)
    return dataset

In [16]:
dataset = drop_by_activity(dataset=dataset, activity_type='Inconclusive', col_name='Activity')

In [17]:
show_activity_distribution(dataset=dataset, label='Activity')

Total dataset
╒════════════════╤══════════╤════════════╤════════════════╤═══════════════╕
│                │   Active │   Inactive │   Inconclusive │   Unspecified │
╞════════════════╪══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 2873     │  25        │              0 │     1071      │
├────────────────┼──────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │   72.386 │   0.629882 │              0 │       26.9841 │
╘════════════════╧══════════╧════════════╧════════════════╧═══════════════╛


## Loại bỏ những chất không có IC50

Những chất không có IC50 là những chất có IC50 trống hoặc bằng 0

In [18]:
# Replace 0 and blank values with NaN
dataset['IC50(uM)'].replace(['0', ''], float('nan'), inplace=True)
dataset = dataset.dropna(subset=['IC50(uM)'])

In [19]:
len(dataset)

3956

In [20]:
show_activity_distribution(dataset=dataset, label='Activity')

Total dataset
╒════════════════╤═══════════╤════════════╤════════════════╤═══════════════╕
│                │    Active │   Inactive │   Inconclusive │   Unspecified │
╞════════════════╪═══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 2873      │  17        │              0 │     1066      │
├────────────────┼───────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │   72.6239 │   0.429727 │              0 │       26.9464 │
╘════════════════╧═══════════╧════════════╧════════════════╧═══════════════╛


In [21]:
non_float_rows = find_non_float_ic50(dataset=dataset, ic50_col_name='IC50(uM)')
non_float_rows.head()

,CID,SMILES,IC50(uM),Activity
16,3810,CN(C)C1=CC=C(C=C1)C(=O)NCCCCC(=O)NO,>10,Unspecified
17,3810,CN(C)C1=CC=C(C=C1)C(=O)NCCCCC(=O)NO,>10,Unspecified
18,3811,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCC(=O)NO,>10,Unspecified
48,4996,C1=CC(=CN=C1)NC(=O)CCCCCCC(=O)NO,>10,Unspecified
273,211808,C1=CC=C(C=C1)CNCCC(=O)NO,>100,Unspecified


In [22]:
show_activity_distribution(dataset=non_float_rows, label='Activity')

Total dataset
╒════════════════╤══════════╤════════════╤════════════════╤═══════════════╕
│                │   Active │   Inactive │   Inconclusive │   Unspecified │
╞════════════════╪══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │        0 │          0 │              0 │           742 │
├────────────────┼──────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │        0 │          0 │              0 │           100 │
╘════════════════╧══════════╧════════════╧════════════════╧═══════════════╛


## Ghép chất và tính trung bình từng chất

__Kiểm tra nhãn mâu thuẫn__

In [23]:
check_label_intersection(dataset=dataset, col_name='Activity')

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 0 │                     0 │                   74 │                       0 │                      2 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


Cách xử lý:
- Nhãn mới: mốc 1uM, nếu lớn hơn thì là Inactive, nếu nhỏ hơn thì là Active. 
- Với những chất Active, ta sẽ thực hiện việc ghép chất xong tính trung bình bình thường, sau đó gán nhãn mới.
- Với những chất Inactive, ta sẽ thực hiện việc ghép chất xong tính trung bình bình thường, sau đó gán nhãn mới.
- Với những chất Unspecified, ta chuyển thẳng nhãn mới thành Inactive

__Active row__

In [24]:
def get_new_data_by_activity(dataset, activity, col_name, new_columns):
    #Handling the active rows first
    data_rows = dataset.loc[dataset[col_name] == activity]
    new_rows = pd.DataFrame(columns=new_columns)
    for cid in data_rows.CID.unique():
        sub_dataset = data_rows.loc[data_rows.CID == cid]
        avg_ic50, dup_count = 0, 0
        for index, row in sub_dataset.iterrows():
            avg_ic50 = avg_ic50 + float(row["IC50(uM)"])/len(sub_dataset)
            dup_count = dup_count + 1
        new_rows.loc[len(new_rows)] = [row.CID, row.SMILES ,avg_ic50, activity,
                                            {True: 'Active', False: 'Inactive'} [avg_ic50 < 1],
                                            dup_count]
    return new_rows

In [25]:
new_columns = ["CID", "SMILES", "AVG_IC50_uM", "FIRST_LABEL", "FINAL_LABEL", "DUPLICATE_COUNTS"]

In [26]:
new_active_rows = get_new_data_by_activity(dataset=dataset, activity='Active', col_name='Activity', new_columns=new_columns)
print(len(new_active_rows))
new_active_rows.head()

1601


,CID,SMILES,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,2746,CC(=O)NC1=CC=C(C=C1)C(=O)NC2=CC=CC=C2N,2.232250,Active,Inactive,12
1,3811,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCC(=O)NO,0.244000,Active,Active,1
2,3812,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCCCC(=O)NO,1.308333,Active,Inactive,3
3,3994,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCCC(=O)NO,1.255333,Active,Inactive,3
4,4261,C1=CC=C(C(=C1)N)NC(=O)C2=CC=C(C=C2)CNC(=O)OCC3...,1.526116,Active,Inactive,19


__Inactive rows__

In [27]:
inactive_cid_unique = dataset.loc[dataset['Activity'] == 'Inactive']
len(inactive_cid_unique.CID.unique())

15

In [28]:
new_inactive_rows = get_new_data_by_activity(dataset=dataset, activity='Inactive', col_name='Activity', new_columns=new_columns)
print(len(new_inactive_rows))
new_inactive_rows.head()

15


,CID,SMILES,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,6917365,COC(=O)[C@H](CC1=CC=CC=C1)N2C(=C(N=N2)C3=CC=CC...,0.0,Inactive,Active,1
1,60198344,CN1C=C(C=N1)/C=C/C(=O)NC2=CC=CC=C2N,15.0,Inactive,Inactive,2
2,60198346,CC1=NC(=CO1)/C=C/C(=O)NC2=CC=CC=C2N,20.0,Inactive,Inactive,1
3,60198412,CC1=NOC(=C1)/C=C/C(=O)NC2=CC=CC=C2N,15.0,Inactive,Inactive,2
4,60198482,CN1C=C(C=N1)/C=C/C(=O)NC2=C(C=C(C=C2)F)N,20.0,Inactive,Inactive,1


__Unspecified rows__

In [29]:
def get_new_data_for_unspecfied(dataset, col_name, new_columns):
    #Handling the active rows first
    data_rows = dataset.loc[dataset[col_name] == 'Unspecified']
    new_rows = pd.DataFrame(columns=new_columns)
    for cid in data_rows.CID.unique():
        sub_dataset = data_rows.loc[data_rows.CID == cid]
        dup_count = 0
        for index, row in sub_dataset.iterrows():
            dup_count = dup_count + 1
        new_rows.loc[len(new_rows)] = [row.CID, row.SMILES ,'', 'Unspecified', 'Inactive', dup_count]
    return new_rows

In [30]:
new_unspecified_rows = get_new_data_for_unspecfied(dataset=dataset, col_name='Activity', new_columns=new_columns)
print(len(new_unspecified_rows))
new_unspecified_rows.head()

674


,CID,SMILES,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,264,CCCC(=O)O,,Unspecified,Inactive,1
1,3121,CCCC(CCC)C(=O)O,,Unspecified,Inactive,2
2,3810,CN(C)C1=CC=C(C=C1)C(=O)NCCCCC(=O)NO,,Unspecified,Inactive,2
3,3811,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCC(=O)NO,,Unspecified,Inactive,1
4,4775,C1=CC=C(C=C1)CCCC(=O)O,,Unspecified,Inactive,1


__Combine all three to create a new data__

In [31]:
new_data = pd.DataFrame(columns=new_columns)
new_data = pd.concat([new_data, new_active_rows], axis=0)
new_data = pd.concat([new_data, new_inactive_rows], axis=0)
new_data = pd.concat([new_data, new_unspecified_rows], axis=0)
print(len(new_active_rows), len(new_inactive_rows), len(new_unspecified_rows), len(new_data))

1601 15 674 2290


In [32]:
new_data.head()

,CID,SMILES,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,2746,CC(=O)NC1=CC=C(C=C1)C(=O)NC2=CC=CC=C2N,2.23225,Active,Inactive,12
1,3811,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCC(=O)NO,0.244,Active,Active,1
2,3812,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCCCC(=O)NO,1.308333,Active,Inactive,3
3,3994,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCCC(=O)NO,1.255333,Active,Inactive,3
4,4261,C1=CC=C(C(=C1)N)NC(=O)C2=CC=C(C=C2)CNC(=O)OCC3...,1.526116,Active,Inactive,19


In [33]:
show_activity_distribution(dataset=new_data, label='FINAL_LABEL')

Total dataset
╒════════════════╤═══════════╤════════════╤════════════════╤═══════════════╕
│                │    Active │   Inactive │   Inconclusive │   Unspecified │
╞════════════════╪═══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 1087      │  1203      │              0 │             0 │
├────────────────┼───────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │   47.4672 │    52.5328 │              0 │             0 │
╘════════════════╧═══════════╧════════════╧════════════════╧═══════════════╛


In [34]:
show_activity_distribution(dataset=new_data, label='FIRST_LABEL')

Total dataset
╒════════════════╤═══════════╤════════════╤════════════════╤═══════════════╕
│                │    Active │   Inactive │   Inconclusive │   Unspecified │
╞════════════════╪═══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 1601      │  15        │              0 │      674      │
├────────────────┼───────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │   69.9127 │   0.655022 │              0 │       29.4323 │
╘════════════════╧═══════════╧════════════╧════════════════╧═══════════════╛


In [35]:
check_label_intersection(dataset=new_data, col_name='FIRST_LABEL')

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 0 │                     0 │                   74 │                       0 │                      2 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


In [36]:
check_label_intersection(dataset=new_data, col_name='FINAL_LABEL')

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                44 │                     0 │                    0 │                       0 │                      0 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


In [37]:
new_data.to_excel('../../data_for_modeling/filter_data/all_data/HDAC2_all_data_before_canon_smiles.xlsx')

# Lọc trùng sử dụng Cannon SMILES

In [38]:
active_rows = new_data.loc[new_data['FIRST_LABEL'] == "Active"]
unspecified_rows = new_data.loc[new_data['FIRST_LABEL'] == "Unspecified"]
cid_active = active_rows.loc[:, 'CID']
cid_unspec = unspecified_rows.loc[:, 'CID']
ac_unspec_cid = np.intersect1d(cid_active, cid_unspec)
ac_unspec_cid

array([3811, 4996, 5311, 298878, 403373, 444732, 9800555, 9862144,
       9943996, 9965141, 10902481, 10967421, 10968817, 11086699, 11747496,
       11810423, 15986509, 24757913, 24938735, 24944851, 24944855,
       24944856, 24944935, 24945115, 24945204, 24945368, 24945689,
       25155955, 45140150, 45140153, 45140347, 45140348, 45140349,
       46887439, 46887476, 49850262, 56650312, 62706005, 62706006,
       62706167, 62706168, 72734543, 72946782, 102514716, 126650038,
       127031793, 130345472, 134283799, 137634156, 141733188, 146317396,
       148289288, 155513116, 155515488, 155518327, 155518496, 155518725,
       155521283, 155525782, 155532640, 155538765, 155545380, 155564400,
       155907155, 156018320, 156021459, 162658937, 162660161, 162661061,
       162665019, 162667559, 162673723, 164625443, 164625922],
      dtype=object)

In [39]:
def make_canonical_smiles(smiles):
    mols = [Chem.MolFromSmiles(smi) for smi in smiles]
    smiles = [Chem.MolToSmiles(mol) for mol in mols]
    return smiles

In [40]:
cannon_smiles = make_canonical_smiles(dataset.SMILES)
dataset['SMILES'] = cannon_smiles

In [41]:
smiles_data = ["OC(=O)C(Br)(Cl)N", "ClC(Br)(N)C(=O)O", "O=C(O)C(N)(Br)Cl", "[CH3][CH2][OH]"]
smiles_data = make_canonical_smiles(smiles=smiles_data)
smiles_data

['NC(Cl)(Br)C(=O)O', 'NC(Cl)(Br)C(=O)O', 'NC(Cl)(Br)C(=O)O', 'CCO']

In [42]:
def remove_dup_smiles(dataset, smile_col_name):
    unique_result = pd.DataFrame(columns=dataset.columns)
    dup_result = pd.DataFrame(columns=dataset.columns)
    error_result = pd.DataFrame(columns=dataset.columns)
    #Start iteration
    unique_smiles = dataset[smile_col_name].unique()
    for smile in unique_smiles:
        sub_dataset = new_data.loc[new_data.SMILES == smile]
        if(len(sub_dataset) == 1):
            unique_result = pd.concat([unique_result, sub_dataset], axis=0)
        elif(len(sub_dataset) > 1):
            dup_result = pd.concat([dup_result, sub_dataset], axis=0)
        else:
            error_result = pd.concat([error_result, sub_dataset], axis=0)
    return unique_result, dup_result, error_result

In [43]:
unique_result, dup_result, error_result = remove_dup_smiles(dataset=new_data, smile_col_name='SMILES')

In [44]:
print(len(unique_result), len(dup_result), len(error_result))

2138 152 0


In [45]:
new_data = unique_result
print(len(new_data))
new_data.head()

2138


,CID,SMILES,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,2746,CC(=O)NC1=CC=C(C=C1)C(=O)NC2=CC=CC=C2N,2.23225,Active,Inactive,12
2,3812,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCCCC(=O)NO,1.308333,Active,Inactive,3
3,3994,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCCC(=O)NO,1.255333,Active,Inactive,3
4,4261,C1=CC=C(C(=C1)N)NC(=O)C2=CC=C(C=C2)CNC(=O)OCC3...,1.526116,Active,Inactive,19
6,5173,C(CCCC(=O)NO)CCC(=O)NO,8.23,Active,Inactive,1


In [46]:
unique_result.head()

,CID,SMILES,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,2746,CC(=O)NC1=CC=C(C=C1)C(=O)NC2=CC=CC=C2N,2.23225,Active,Inactive,12
2,3812,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCCCC(=O)NO,1.308333,Active,Inactive,3
3,3994,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCCC(=O)NO,1.255333,Active,Inactive,3
4,4261,C1=CC=C(C(=C1)N)NC(=O)C2=CC=C(C=C2)CNC(=O)OCC3...,1.526116,Active,Inactive,19
6,5173,C(CCCC(=O)NO)CCC(=O)NO,8.23,Active,Inactive,1


In [47]:
dup_result.head()

,CID,SMILES,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
1,3811,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCC(=O)NO,0.244,Active,Active,1
3,3811,CN(C)C1=CC=C(C=C1)C(=O)NCCCCCC(=O)NO,,Unspecified,Inactive,1
5,4996,C1=CC(=CN=C1)NC(=O)CCCCCCC(=O)NO,4.05,Active,Inactive,2
5,4996,C1=CC(=CN=C1)NC(=O)CCCCCCC(=O)NO,,Unspecified,Inactive,1
8,5311,C1=CC=C(C=C1)NC(=O)CCCCCCC(=O)NO,0.329001,Active,Active,209


# Đánh giá bộ dữ liệu

In [48]:
check_label_intersection(dataset=new_data, col_name='FIRST_LABEL')

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 0 │                     0 │                    0 │                       0 │                      0 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


In [49]:
check_label_intersection(dataset=new_data, col_name='FINAL_LABEL')

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 0 │                     0 │                    0 │                       0 │                      0 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


In [50]:
print("Label for first label:")
show_activity_distribution(dataset=new_data, label='FIRST_LABEL')
print()
print("Label for final label:")
show_activity_distribution(dataset=new_data, label='FINAL_LABEL')

Label for first label:
Total dataset
╒════════════════╤═══════════╤════════════╤════════════════╤═══════════════╕
│                │    Active │   Inactive │   Inconclusive │   Unspecified │
╞════════════════╪═══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 1527      │  13        │              0 │      598      │
├────────────────┼───────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │   71.4219 │   0.608045 │              0 │       27.9701 │
╘════════════════╧═══════════╧════════════╧════════════════╧═══════════════╛

Label for final label:
Total dataset
╒════════════════╤═══════════╤════════════╤════════════════╤═══════════════╕
│                │    Active │   Inactive │   Inconclusive │   Unspecified │
╞════════════════╪═══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 1043      │  1095      │              0 │             0 │
├────────────────┼───────────┼────────────┼────────────────┼───────────────┤
│

In [52]:
with pd.ExcelWriter('../../data_for_modeling/filter_data/all_data/HDAC2_all_final_data.xlsx', engine='openpyxl') as writer:
    writer.book = writer.book 
    dataset_c.to_excel(writer, sheet_name='original_data', index=True)
    new_data.to_excel(writer, sheet_name='filter_data', index=True)
    dup_result.to_excel(writer, sheet_name='duplicate_smiles', index=True)

/tmp/ipykernel_190939/2965386203.py:2: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = writer.book
